In [258]:
from sklearn.model_selection import train_test_split
import pandas as pd
from torch import nn
from transformers import AutoTokenizer, AutoModel
import torch


In [259]:
df = pd.read_csv("./data/raw/train.csv")
df_test = pd.read_csv("./data/raw/test.csv")


In [260]:
train.head()


,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate,tokenized
2992,080f938a32768670,Typo'? \n\nYour first link here is to a commen...,0,0,0,0,0,0,"{'input_ids': tensor([[ 101, 5939, 6873, 1005,..."
14654,26a8f94d79e33e4c,"Welcome!\n\nHello, Reimmichl-212, and welcome ...",0,0,0,0,0,0,"{'input_ids': tensor([[ 101, 6160, 999, 7..."
21081,379d09519e42361e,"figures you have aspergers, pedantic little fu...",1,0,1,0,1,0,"{'input_ids': tensor([[ 101, 4481, 2017, 2..."
81638,da60c44ff4f75b6e,Anna Frodesiak motherfucker bitch,1,0,1,0,1,0,"{'input_ids': tensor([[ 101, 4698, 10424, 19..."
4505,0bfdc705076c1c77,"""\n\nResponse to your recent edit at Loyola Ac...",0,0,0,0,0,0,"{'input_ids': tensor([[ 101, 1000, 3433, 2..."


In [261]:
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
model = AutoModel.from_pretrained("distilbert-base-uncased")


Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_projector.weight', 'vocab_layer_norm.bias', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_transform.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [262]:
def extract_target(df):
    return (df['toxic severe_toxic obscene threat insult identity_hate'.split()].to_numpy())


def tokenize_data(df):
    return tokenizer(list(df["comment_text"]),  padding="max_length", truncation=True, return_tensors='pt', return_attention_mask=False)['input_ids']


In [263]:
# df = pd.read_csv('./data/checkpoint_1.csv')

train, val = train_test_split(df, test_size=0.3, train_size=0.7, shuffle=True)
X_train = tokenize_data(train)
X_val = tokenize_data(val)
y_train = torch.Tensor(extract_target(train))
y_val = torch.Tensor(extract_target(val))


In [264]:
class Classifier(nn.Module):
    def __init__(self, input_dim) -> None:
        super().__init__()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(input_dim, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 6),
        )

    def forward(self, x):
        logits = self.linear_relu_stack(x)
        return logits


In [265]:
input_dim = 768
learning_rate = 0.003
batch_size = 32


In [269]:
from torch.utils.data import DataLoader, TensorDataset
device = torch.device('cuda')
dataset = TensorDataset(X_train.to(device), y_train.to(device))  # type: ignore
dataloader = DataLoader(dataset, batch_size=batch_size)

classifier = Classifier(input_dim).to(device)
loss_fn = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(classifier.parameters(), lr=learning_rate)


AssertionError: Torch not compiled with CUDA enabled

In [ ]:
(X, y) = next(iter(dataloader))
y


In [268]:
(X, y) = next(iter(dataloader))
embed = model(X).last_hidden_state[:, 0]
pred = classifier(embed)
loss_fn(pred, y)


tensor(0.6917, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)

In [ ]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
        embed = model(X).last_hidden_state[:, 0]
        pred = classifier(embed)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 512 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")
            test_loss, correct = 0, 0
            with torch.no_grad():
                for X, y in dataloader:
                    pred = model(X)
                    test_loss += loss_fn(pred, y).item()
                    correct += (pred.argmax(1) ==
                                y).type(torch.float).sum().item()


# def test_loop(dataloader, model, loss_fn):
#     size = len(dataloader.dataset)
#     num_batches = len(dataloader)
#     test_loss, correct = 0, 0

#     with torch.no_grad():
#         for X, y in dataloader:
#             pred = model(X)
#             test_loss += loss_fn(pred, y).item()
#             correct += (pred.argmax(1) == y).type(torch.float).sum().item()

#     test_loss /= num_batches
#     correct /= size
#     print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")
